In [1]:
# Import Modules
import pandas as pd
import numpy  as np
from scipy import stats
from scipy.stats import uniform
from scipy.stats import sem
import matplotlib.pyplot as plt  

In [2]:
your_path = './Output_1/'

In [9]:
# initialization module
 
Price_Per_Unit   = 27.0
Cost_Per_Unit    = 15.50
Salvage_Per_Unit = 2.50

# list_of_orders = [40]                 ## quantity ordered for the pilot run
list_of_orders = [20,30,40,45,50,60]
length = len(list_of_orders)

bound = 10                           ## selected by the analyst 
confidence = 0.95                      ## selected by the analyst 

In [5]:
# historical daily demand
daily_demand = [5, 10, 40, 45, 50, 55, 60]
# discrete probabilities for daily demand
pr1, pr2, pr3, pr4, pr5, pr6, pr7 = 0.1, 0.2, 0.3, 0.2, 0.1, 0.05, 0.05
 
pr1 = round(pr1, 2)
pr2 = round(pr1 + pr2,2)
pr3 = round(pr2 + pr3,2)
pr4 = round(pr3 + pr4,2)
pr5 = round(pr4 + pr5,2)
pr6 = round(pr5 + pr6,2)
pr7 = round(pr6 + pr7,2)

list_of_probs = [pr1, pr2, pr3, pr4, pr5, pr6, pr7]

df1 = pd.DataFrame(daily_demand, columns = ['demand'])
df2 = pd.DataFrame(list_of_probs,columns = ['range'])
 
df_demand = pd.concat([df1, df2], axis = 1)

In [6]:
column_labels = ["Order Quantity", "Mean","Std.Dev","Var.",
                 "Std. Error","Median", "Skewness", "Kurtosis",
                 "CI Half Width", "CI LL", 'CI UL']
df = pd.DataFrame(columns = column_labels)

In [14]:
#....
Number_of_Replications = 100
list_numb_of_samples = []
for j in range(length):
    list_of_profits = []
    for run in range(Number_of_Replications):
        # uniform distribution for a random variate
        r_v = uniform.rvs(size=1)
             
        for i,row in df_demand.iterrows():
            probab = df_demand.loc[i, 'range']
             
            if r_v < probab:
                Qty_Demand = df_demand.loc[i, 'demand']
                break
        
        Qty_Ordered = 40 ## list_of_orders[j]
        
        Qty_Sold = np.minimum(Qty_Demand, Qty_Ordered)
        Qty_Left = np.maximum(0, Qty_Ordered - Qty_Demand)
         
        Revenue_per_Sales = Qty_Sold * Price_Per_Unit        
        Revenue_per_Salvage = Qty_Left * Salvage_Per_Unit        
        Cost_per_Order =  Qty_Ordered * Cost_Per_Unit
        
        Profit = Revenue_per_Sales +  Revenue_per_Salvage - Cost_per_Order        
        list_of_profits.append(Profit)
    
    media = np.mean(list_of_profits)
    stand = np.std(list_of_profits)
                
    dof  = Number_of_Replications - 1    
    t_crit = np.abs(stats.t.ppf((1-confidence)/2,dof))

    numb_of_samples = int((stand *t_crit / bound) ** 2)
    list_numb_of_samples.append(numb_of_samples)
    # print('')
    # print(' The number of samples is %3s' %numb_of_samples)
print(f' The number of samples is min = {min(list_numb_of_samples)}, max = {max(list_numb_of_samples)}, mean = {np.mean(list_numb_of_samples)}')

 The number of samples is min = 4295, max = 5605, mean = 5114.666666666667
